In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

In [2]:
BASE_URL = "https://finance.naver.com/sise/sise_market_sum.naver"
# https://finance.naver.com/sise/sise_market_sum.naver?&page=1
num_last_page = 0

In [3]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(BASE_URL)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64_m1.zip
[WDM] - Driver has been saved in cache [/Users/krc/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61]


### 1. 마지막 페이지 번호 찾기

In [4]:
xpath = "//*[contains(text(),'맨뒤')]"
search_xpath = driver.find_element(by=By.XPATH, value=xpath)
num_last_page = int(str(search_xpath.get_attribute('href'))[-2:])
# //*[@id="contentarea"]/div[3]/table[2]/tbody/tr/td[12]/a

In [5]:
print(num_last_page)
driver.close()

37


### 2. 페이지 번호 변경하면서 데이터 긁어오기

In [6]:
columns = ['종목명', '현재가', '전일비', '등락률', '액면가', '시가총액', '상장주식수', '외국인비율', '거래량', 'PER', 'ROE', '토론실']
tables = pd.DataFrame(columns=columns)
# tables = tables.reset_index()

In [7]:
tables

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실


In [186]:
# res = requests.get(BASE_URL+'?&page=1')
# time.sleep(1)
# # print(res.status_code)
# soup = BeautifulSoup(res.content.decode("euc-kr", "replace"), "html.parser")
# soup_table = soup.select_one("div.box_type_l")
# table = pd.read_html(str(soup_table))
# table = table[0]

In [187]:
# table

In [188]:
# pd.concat([tables, table], axis=0, sort=False)

In [8]:
for i in range(1, num_last_page+1):
    try:
        res = requests.get(BASE_URL+f"?&page={i}")
        time.sleep(1)
        # print(res.status_code)
        soup = BeautifulSoup(res.content.decode("euc-kr", "replace"), "html.parser")
        soup_table = soup.select_one("div.box_type_l")
        table = pd.read_html(str(soup_table))
        table = table[0]
        # print(table)
        tables = pd.concat([tables, table], axis=0, sort=False)
        # print(tables)

    except:
        print("Something went wrong.")

### Check results

In [9]:
tables.head()

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,삼성전자,65600.0,100.0,+0.15%,100.0,3916177.0,5969783.0,50.54,2849590.0,10.3,13.92,NaN,1.0
2,LG에너지솔루션,432000.0,0.0,0.00%,500.0,1010880.0,234000.0,3.22,20935.0,137.67,10.68,NaN,2.0
3,SK하이닉스,106000.0,2000.0,+1.92%,5000.0,771682.0,728002.0,49.77,724811.0,7.29,16.84,NaN,3.0
4,삼성바이오로직스,830000.0,8000.0,+0.97%,2500.0,590744.0,71174.0,10.52,6702.0,115.89,8.21,NaN,4.0


In [10]:
tables.tail()

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실,N
4,KBSTAR 200산업재,12285.0,50.0,-0.41%,0.0,10.0,80.0,0.0,60.0,NaN,NaN,NaN,1804.0
5,KBSTAR 200생활소비재,6925.0,30.0,-0.43%,0.0,10.0,140.0,0.0,57.0,NaN,NaN,NaN,1805.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2925 entries, 0 to 8
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종목명     1805 non-null   object 
 1   현재가     1805 non-null   object 
 2   전일비     1805 non-null   object 
 3   등락률     1805 non-null   object 
 4   액면가     1805 non-null   object 
 5   시가총액    1805 non-null   object 
 6   상장주식수   1805 non-null   object 
 7   외국인비율   1805 non-null   object 
 8   거래량     1805 non-null   object 
 9   PER     931 non-null    object 
 10  ROE     795 non-null    object 
 11  토론실     0 non-null      object 
 12  N       1805 non-null   float64
dtypes: float64(1), object(12)
memory usage: 319.9+ KB


In [222]:
tables.dropna(how='all')
tables.drop("토론실", axis=1, inplace=True)
tables.drop("N", axis=1, inplace=True)

In [223]:
tables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2923 entries, 0 to 6
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     1803 non-null   object
 1   현재가     1803 non-null   object
 2   전일비     1803 non-null   object
 3   등락률     1803 non-null   object
 4   액면가     1803 non-null   object
 5   시가총액    1803 non-null   object
 6   상장주식수   1803 non-null   object
 7   외국인비율   1803 non-null   object
 8   거래량     1803 non-null   object
 9   PER     932 non-null    object
 10  ROE     795 non-null    object
dtypes: object(11)
memory usage: 274.0+ KB


In [224]:
tables["PBR"] = (tables.PER * tables.ROE / 100)

In [225]:
tables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2923 entries, 0 to 6
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     1803 non-null   object
 1   현재가     1803 non-null   object
 2   전일비     1803 non-null   object
 3   등락률     1803 non-null   object
 4   액면가     1803 non-null   object
 5   시가총액    1803 non-null   object
 6   상장주식수   1803 non-null   object
 7   외국인비율   1803 non-null   object
 8   거래량     1803 non-null   object
 9   PER     932 non-null    object
 10  ROE     795 non-null    object
 11  PBR     793 non-null    object
dtypes: object(12)
memory usage: 296.9+ KB


In [239]:
# tables[tables["종목명"]=="롯데정밀화학"]

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,PBR
67,롯데정밀화학,82400.0,1200.0,-1.44%,5000.0,21259.0,25800.0,20.38,32715.0,3.7,30.43,1.12591


In [244]:
filtered_stock = tables[(tables.PBR > 0) & (tables.PBR < 0.6) & (tables.PER > 0) & (tables.PER < 12)]

In [245]:
filtered_stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 18 to 43
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     137 non-null    object
 1   현재가     137 non-null    object
 2   전일비     137 non-null    object
 3   등락률     137 non-null    object
 4   액면가     137 non-null    object
 5   시가총액    137 non-null    object
 6   상장주식수   137 non-null    object
 7   외국인비율   137 non-null    object
 8   거래량     137 non-null    object
 9   PER     137 non-null    object
 10  ROE     137 non-null    object
 11  PBR     137 non-null    object
dtypes: object(12)
memory usage: 13.9+ KB


In [246]:
filtered_stock.to_excel("stocks_under_PBR_1.2.xlsx", sheet_name="kospi", index=False)